# Step 1: Create a Simple SQLite Database

In [1]:
import sqlite3

# Step 1: Connect to the SQLite database (students.db)
# If the database does not exist, it will be created automatically.
conn = sqlite3.connect('students.db')
cursor = conn.cursor()

# Step 2: Create the students table if it doesn't exist
# This table will have three columns: name, branch, and cgpa.
cursor.execute('''
CREATE TABLE IF NOT EXISTS students (
    name TEXT,
    branch TEXT,
    cgpa REAL
)
''')

# Step 3: Insert sample data into the students table
# We are adding multiple records for different students along with their branches and CGPAs.
sample_data = [
    ('Ishaan Gupta', 'EE', 9.7),
    ('Ayaan Patel', 'CE', 8.45),
    ('Vihaan Mishra', 'CSE', 8.88),
    ('Aarav Sharma', 'AI', 6.81),
    ('Ayaan Sharma', 'CE', 9.86),
    ('Ayaan Reddy', 'CSE', 7.4),
    ('Ishaan Verma', 'CE', 7.74),
    ('Vivaan Verma', 'ME', 6.59),
    ('Reyansh Mishra', 'CE', 7.97),
    ('Ayaan Sharma', 'ME', 9.05),
    ('Aarav Gupta', 'ME', 8.47),
    ('Reyansh Mishra', 'ME', 7.84),
    ('Vivaan Singh', 'CSE', 6.41),
    ('Arjun Reddy', 'CE', 8.6),
    ('Aditya Reddy', 'CE', 7.04),
    ('Ayaan Reddy', 'EE', 6.91)
]

# Step 4: Insert the sample data into the table
# We use executemany to insert multiple rows at once.
cursor.executemany('INSERT INTO students (name, branch, cgpa) VALUES (?, ?, ?)', sample_data)

# Step 5: Commit the changes to the database
# This saves all the changes made to the database.
conn.commit()

# Step 6: Print a confirmation message
print("Database and sample data ready!")

# Step 7: Close the connection to the database
conn.close()


Database and sample data ready!


## Step 2: Basic NL2SQL function

In [3]:
def generate_sql(user_input):
    # Step 1: Convert the user input to lowercase for case-insensitive comparison
    user_input = user_input.lower()

    # Step 2: Check if the user wants to list all students
    # If the user asks for "all students" or "list students", we return the SQL to get all student records
    if "all students" in user_input or "list students" in user_input:
        return "SELECT * FROM students;"

    # Step 3: Check if the user is asking for students with a grade above a certain value
    # Look for the phrase "students above" and extract the number after it
    elif "students above" in user_input:
        # Split the input into individual words
        words = user_input.split()
        # Loop through the words to find the position of "above"
        for i, word in enumerate(words):
            if word == "above":
                try:
                    # Try to convert the next word into an integer (the grade threshold)
                    number = int(words[i+1])
                    # Return SQL to fetch students with a grade greater than the extracted number
                    return f"SELECT * FROM students WHERE grade > {number};"
                except:
                    # If there's an error (e.g., no number provided), return None
                    return None

    # Step 4: Check if the user wants the average grade of all students
    # If the user asks for the average grade, return the SQL to calculate the average grade
    elif "average grade" in user_input:
        return "SELECT AVG(grade) FROM students;"

    # Step 5: Check if the user wants the count of all students
    # If the user asks for the count of students, return the SQL to count them
    elif "count students" in user_input:
        return "SELECT COUNT(*) FROM students;"

    # Step 6: If none of the above conditions are met, return None
    else:
        return None  # If nothing matched, return None


## Step 3: Execute SQL and Fetch Result

In [5]:
def fetch_sql_query(user_input):
    # Step 1: Convert the user input to lowercase for case-insensitive comparison
    user_input = user_input.lower()

    # Step 2: Handle common synonyms and spelling errors in user input
    synonyms = {
        'aie': 'ai', 'csee': 'cse', 'mechanical': 'me', 'mech': 'me', 'civil': 'ce', 'electrical': 'ee',
        'artificial intelligence': 'ai', 'computer science': 'cse', 'mechanical engineering': 'me',
        'civil engineering': 'ce', 'electrical engineering': 'ee',
        'show': 'list', 'details': 'list', 'data': 'list'
    }
    for wrong, correct in synonyms.items():
        user_input = user_input.replace(wrong, correct)

    # Step 3: Extract numbers (like CGPA values) from user input
    numbers = [float(s) for s in user_input.split() if s.replace('.', '', 1).isdigit()]

    # Step 4: Query for overall topper
    if "topper" in user_input and "branch" not in user_input:
        return "SELECT name, branch, cgpa FROM students ORDER BY cgpa DESC LIMIT 1"

    # Step 5: Query for topper in specific branch
    if "topper" in user_input and "branch" in user_input:
        for branch in ["ai", "cse", "me", "ce", "ee"]:
            if branch in user_input:
                return f"SELECT name, branch, cgpa FROM students WHERE branch = '{branch.upper()}' ORDER BY cgpa DESC LIMIT 1"

    # Step 6: Average CGPA (overall or branch-wise)
    if "average" in user_input or "avg" in user_input:
        for branch in ["ai", "cse", "me", "ce", "ee"]:
            if branch in user_input:
                return f"SELECT AVG(cgpa) FROM students WHERE branch = '{branch.upper()}'"
        return "SELECT AVG(cgpa) FROM students"

    # Step 7: Return list of students per department instead of just count
    if ("count" in user_input or "how many" in user_input) and "student" in user_input:
        return "SELECT name, branch, cgpa FROM students ORDER BY branch"

    # Step 8: Students with CGPA in a range
    if ("cgpa between" in user_input or "cgpa from" in user_input) and len(numbers) >= 2:
        return f"SELECT name, branch, cgpa FROM students WHERE cgpa BETWEEN {numbers[0]} AND {numbers[1]}"

    # Step 9: Students with CGPA greater than a value
    if ("greater" in user_input or "above" in user_input) and "cgpa" in user_input and numbers:
        return f"SELECT name, branch, cgpa FROM students WHERE cgpa > {numbers[0]}"

    # Step 10: Students with CGPA less than a value
    if ("less" in user_input or "below" in user_input) and "cgpa" in user_input and numbers:
        return f"SELECT name, branch, cgpa FROM students WHERE cgpa < {numbers[0]}"

    # Step 11: Sort students by CGPA
    if "sort" in user_input or "sorted" in user_input:
        return "SELECT name, branch, cgpa FROM students ORDER BY cgpa DESC"

    # Step 12: Students from a specific branch
    for branch in ["ai", "cse", "me", "ce", "ee"]:
        if branch in user_input:
            return f"SELECT name, branch, cgpa FROM students WHERE branch = '{branch.upper()}'"

    # Step 13: Search student by name
    if "student" in user_input or "search" in user_input or "find" in user_input:
        words = user_input.split()
        probable_names = [w.capitalize() for w in words if w.isalpha() and len(w) > 2]
        if probable_names:
            return f"SELECT name, branch, cgpa FROM students WHERE name LIKE '%{probable_names[-1]}%'"

    # Step 15: Default fallback query
    return "SELECT name, branch, cgpa FROM students"


## get_response() function (to generate a nice bot reply)

In [12]:
def get_response(user_input):
    try:
        # Step 1: Get the SQL query based on user input using the fetch_sql_query function
        query = fetch_sql_query(user_input)

        # Step 2: Execute the query on the database using the cursor
        cursor.execute(query)
        
        # Step 3: Fetch all results from the executed query
        results = cursor.fetchall()

        # Step 4: If no results are returned, provide a random funny reply
        if not results:
            funny_replies = [
                "Nothing found! ",
                "I searched, but found nothing! ",
                "Looks like this is not in the data! ",
                "Sorry, no data found! "
            ]
            import random
            # Return a random funny reply if no results are found
            return random.choice(funny_replies)

        # Step 5: Initialize an empty string to build the response
        response = ""
        
        # Step 6: If the query involves calculating the average (AVG), format the response
        if "avg" in query.lower():
            # Round the average CGPA to two decimal places and include it in the response
            response = f" Average CGPA: {round(results[0][0], 2)}"
        
        # Step 7: If the query involves counting students per branch, format the response
        elif "count" in query.lower():
            # Loop through the result rows and format the count of students per branch
            for row in results:
                response += f"Branch: {row[0]}, Students: {row[1]}\n"
        
        # Step 8: For other queries, format the response with student details (name, branch, CGPA)
        else:
            # Loop through the result rows and include student details in the response
            for row in results:
                response += f"Name: {row[0]}, Branch: {row[1]}, CGPA: {row[2]}\n"
        
        # Step 9: Strip any leading/trailing spaces from the response before returning
        return response.strip()

    except Exception as e:
        # Step 10: Handle any errors by returning the error message
        return f" Error: {e}"


## Step 4:  chatbot_gui.py

In [6]:
import tkinter as tk
from tkinter import scrolledtext
import sqlite3

# ----------------- Function to connect with the database -----------------
def connect_db():
    conn = sqlite3.connect('students.db')
    return conn

# ----------------- Function to generate SQL query based on user input -----------------
def fetch_sql_query(user_input):
    user_input = user_input.lower()

    if "top 5" in user_input or "top five" in user_input:
        return "SELECT name, branch, cgpa FROM students ORDER BY cgpa DESC LIMIT 5"
    
    elif "topper" in user_input:
        return "SELECT name, branch, cgpa FROM students ORDER BY cgpa DESC LIMIT 1"
    
    elif "ai branch" in user_input:
        return "SELECT name, branch, cgpa FROM students WHERE branch = 'AI'"
    
    elif "cse branch" in user_input:
        return "SELECT name, branch, cgpa FROM students WHERE branch = 'CSE'"
    
    elif "me branch" in user_input:
        return "SELECT name, branch, cgpa FROM students WHERE branch = 'ME'"
    
    elif "ce branch" in user_input:
        return "SELECT name, branch, cgpa FROM students WHERE branch = 'CE'"
    
    elif "highest cgpa" in user_input or "top cgpa" in user_input:
        return "SELECT name, branch, cgpa FROM students ORDER BY cgpa DESC LIMIT 1"
    
    elif "lowest cgpa" in user_input or "bottom cgpa" in user_input:
        return "SELECT name, branch, cgpa FROM students ORDER BY cgpa ASC LIMIT 1"

    elif "cgpa more than" in user_input or "cgpa greater than" in user_input:
        numbers = [float(s) for s in user_input.split() if s.replace('.', '', 1).isdigit()]
        if numbers:
            return f"SELECT name, branch, cgpa FROM students WHERE cgpa > {numbers[0]}"
    
    elif "cgpa less than" in user_input or "cgpa below" in user_input:
        numbers = [float(s) for s in user_input.split() if s.replace('.', '', 1).isdigit()]
        if numbers:
            return f"SELECT name, branch, cgpa FROM students WHERE cgpa < {numbers[0]}"
    
    elif "cgpa equal to" in user_input or "cgpa is" in user_input:
        numbers = [float(s) for s in user_input.split() if s.replace('.', '', 1).isdigit()]
        if numbers:
            return f"SELECT name, branch, cgpa FROM students WHERE cgpa = {numbers[0]}"

    else:
        # Default case: fetch all students
        return "SELECT name, branch, cgpa FROM students"

# ----------------- Function to execute the query and prepare the response -----------------
def get_response(user_input):
    try:
        query = fetch_sql_query(user_input)
        conn = connect_db()
        cursor = conn.cursor()
        cursor.execute(query)
        rows = cursor.fetchall()

        response = ""
        for row in rows:
            response += f"Name: {row[0]}, Branch: {row[1]}, CGPA: {row[2]}\n"

        if not response:
            response = "No records found."
        
        conn.close()
        return response.strip()

    except Exception as e:
        return f"Error: {e}"

# ----------------- Function to handle sending message -----------------
def send_message(event=None):
    user_input = entry.get()
    if user_input.strip() == "":
        return
    
    chatlog.config(state=tk.NORMAL)
    chatlog.insert(tk.END, "You: " + user_input + "\n")
    
    bot_response = get_response(user_input)
    chatlog.insert(tk.END, "Bot: " + bot_response + "\n\n")
    
    entry.delete(0, tk.END)
    chatlog.config(state=tk.DISABLED)
    chatlog.yview(tk.END)

# ----------------- GUI Setup -----------------
root = tk.Tk()
root.title("Student Query Chatbot")
root.geometry("600x450")
root.configure(bg="#E6F7FF")

# Chat log where conversation will be shown
chatlog = scrolledtext.ScrolledText(
    root,
    wrap=tk.WORD,
    width=70,
    height=20,
    state=tk.DISABLED,
    bg="#F0F8FF",
    fg="#333333",
    font=("Arial", 11)
)
chatlog.grid(row=0, column=0, columnspan=2, padx=10, pady=10)

# Entry field for user input
entry = tk.Entry(root, width=60, font=("Arial", 11))
entry.grid(row=1, column=0, padx=10, pady=10)

# Send button to send the message
send_button = tk.Button(
    root,
    text="Send",
    width=20,
    command=send_message,
    bg="#4CAF50",
    fg="white",
    font=("Arial", 11, "bold")
)
send_button.grid(row=1, column=1, padx=10, pady=10)

# Binding the Enter key to send message
root.bind('<Return>', send_message)

# Running the GUI loop
root.mainloop()
